In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/give-me-some-credit-dataset/cs-training.csv')

In [ ]:
df.sample(5)

**Column 1**

In [ ]:
df.rename(columns = {df.columns[0]:'ID'}, inplace = True)

In [ ]:
df.info()

**Column 2**

In [ ]:
df['SeriousDlqin2yrs'].unique()

In [ ]:
P = df.groupby(by = 'SeriousDlqin2yrs')['ID'].count().reset_index()

In [ ]:
P.rename(columns = {'ID':'Counts'},inplace = True)

In [ ]:
P['Percent'] = P['Counts']*100/P['Counts'].sum()

In [ ]:
P

In [ ]:
# Or simply 
df['SeriousDlqin2yrs'].value_counts(normalize = True).plot(kind= 'barh')

**Column 3**

In [ ]:
df['RevolvingUtilizationOfUnsecuredLines'].describe()

#it is a percentage column and anything > 1 could be set to NaN

In [ ]:
sns.distplot(df['RevolvingUtilizationOfUnsecuredLines'])

# we have an outlier at approx 50000

In [ ]:
sns.distplot(df['RevolvingUtilizationOfUnsecuredLines'][df['RevolvingUtilizationOfUnsecuredLines'] <=1])

In [ ]:
df['RevolvingUtilizationOfUnsecuredLines'] = df['RevolvingUtilizationOfUnsecuredLines'].apply(lambda x: np.NaN if x >1 else x)

In [ ]:
df['RevolvingUtilizationOfUnsecuredLines'].describe()

In [ ]:
df['RevolvingUtilizationOfUnsecuredLines'].fillna(df['RevolvingUtilizationOfUnsecuredLines'].mean(),inplace = True)

In [ ]:
df['RevolvingUtilizationOfUnsecuredLines'].describe()

**Column 4**

In [ ]:
df.age.describe()

In [ ]:
sns.distplot(df.age)

In [ ]:
# since only a few ages are below 18 and above 90, lets cap them

df.loc[df.age > 90, 'age'] = 90
df.loc[df.age < 18, 'age'] = 18

This coded gets error.

df.age[df.age > 90] = 90

Therefore

df.loc[df.age > 90, 'age'] = 90

you are trying to edit values on a copy of dataframe

In [ ]:
sns.distplot(df.age)

Columns - 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse' are all redundant as it has helped to populate 'SeriousDlqin2yrs'. Hence these cols can be dropped

In [ ]:
df.drop(columns = ['NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse'],inplace = True)

**Param 5**

In [ ]:
df.DebtRatio.describe()

#this is a percentage column and cannot be more than 1.

In [ ]:
sns.distplot(df.DebtRatio)

In [ ]:
df.DebtRatio[df.DebtRatio > 1].describe()

In [ ]:
df.loc[df.DebtRatio > 1, 'DebtRatio'] = np.NaN

In [ ]:
df.DebtRatio.fillna(method = 'ffill', inplace = True)
sns.distplot(df.DebtRatio)

**Param 6**

In [ ]:
df.MonthlyIncome.describe()

In [ ]:
sns.distplot(df.MonthlyIncome)

There are missing values and Max value is too large. Min value of 0 is not ok as finance industry expect a minimum income of 1000.

The max limit is too high (+06)

In [ ]:
# Count of people with income < 1000
df.ID[df.MonthlyIncome < 1000].count()

In [ ]:
#Lets set is as NAN

df.loc[df.MonthlyIncome < 1000, 'MonthlyIncome'] = np.NaN

In [ ]:
sns.distplot(df.MonthlyIncome[df.MonthlyIncome < 50000])  # After trying with a few values. I stopped on 5000.

In [ ]:
# more than 30000 looks unreasonable according to rest of the data. Hence caping greater incoming
df.loc[df.MonthlyIncome > 30000, 'MonthlyIncome'] = 30000

In [ ]:
sns.distplot(df.MonthlyIncome)

In [ ]:
df.MonthlyIncome.fillna(method = 'ffill', inplace = True)

In [ ]:
df.MonthlyIncome.describe()

In [ ]:
sns.distplot(df.MonthlyIncome)

**Param 7**

In [ ]:
df.NumberOfOpenCreditLinesAndLoans.describe()

In [ ]:
sns.distplot(df.NumberOfOpenCreditLinesAndLoans)

In [ ]:
# the distribution indicate that it is continuous upto 30. Hence, let's cap at 30

df.loc[df.NumberOfOpenCreditLinesAndLoans > 30, 'NumberOfOpenCreditLinesAndLoans'] = 30

In [ ]:
sns.distplot(df.NumberOfOpenCreditLinesAndLoans)

**Param 8**

In [ ]:
df.NumberRealEstateLoansOrLines.describe()

In [ ]:
sns.distplot(df.NumberRealEstateLoansOrLines[df.NumberRealEstateLoansOrLines < 10])

In [ ]:
sns.distplot(df.NumberRealEstateLoansOrLines[df.NumberRealEstateLoansOrLines > 10])

In [ ]:
# Comparing above 2 graphs, looks like we can cap value to 5.

df.loc[df.NumberRealEstateLoansOrLines > 5, 'NumberRealEstateLoansOrLines'] = 5

In [ ]:
sns.distplot(df.NumberRealEstateLoansOrLines)

**Param 9**

In [ ]:
df.NumberOfDependents.unique()

In [ ]:
sns.distplot(df.NumberOfDependents.dropna())

In [ ]:
df.NumberOfDependents.describe()

In [ ]:
# There are missing values and dependents more than 5 can be capped to max 5.

df.loc[df.NumberOfDependents > 5, 'NumberOfDependents'] = 5

df.NumberOfDependents.describe()

In [ ]:
# Since proportion of missing is large, imputation using mean is not appropriate as this will change the distribution too much. 
# we will impute the missing values using ffill as this will preserve the mean and standard deviation.

df.NumberOfDependents.fillna(method = 'ffill', inplace = True)
df.NumberOfDependents.describe()

In [ ]:
# Saving the clean file to pickle
pd.to_pickle(df,'CleanCreditFile.pkl')